In [2]:
import pandas as pd
import numpy as np
import GEOparse, re, copy
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
study_info = pd.read_csv("/data/users-workspace/ruchika.sharma/GenePrioritization/GSE_ids.csv")
GSEID = study_info['GSE_id'].values.tolist()
unique_GSE = set(GSEID)
Log_normalised = study_info.loc[study_info['Tag'] == 'log']
Log_normalised_GSE = set(Log_normalised['GSE_id'])
quantile_normalized = study_info.loc[study_info['Tag'] == 'not_log']
quantile_normalized_GSE = set(quantile_normalized['GSE_id'])
raw_signal = study_info.loc[study_info['Tag'] == 'not_quantile/not_log']
raw_signal_GSE = set(raw_signal['GSE_id'])

In [4]:
unique_GSE

{'GSE148004'}

In [5]:
def getGEO(GSEID):
    try:
        gse = GEOparse.get_GEO(geo = GSEID, destdir = "/data/users-workspace/ruchika.sharma", silent=True)
        return gse
    except Exception as e:
        print(GSEID)
        print(e)

In [6]:
def quantileNormalize(df_input):
    df = df_input.copy()
    #compute rank
    dic = {}
    for col in df:
        dic.update({col : sorted(df[col])})
    sorted_df = pd.DataFrame(dic)
    rank = sorted_df.mean(axis = 1).tolist()
    #sort
    for col in df:
        t = np.searchsorted(np.sort(df[col]), df[col])
        df[col] = [rank[i] for i in t]
    return df

In [21]:
def prep_deg_input(gse, gse_id, quantile , log):
#     if gpl is None:
#         raise ValueError("gpl is None")
    #For expression data
    gpl_name = ([key for key in gse.gpls.keys()])[0]
    pivoted_samples = gse.pivot_samples('VALUE')
    samples = gse.pivot_samples("VALUE")
    gse.pivot_samples('VALUE')
    #gse.gpls[gpl_name].table.head(50)
    gse_annotated = pivoted_samples.reset_index().merge(gse.gpls[gpl_name].table[["ID","GENE_SYMBOL"]], left_on='ID_REF', right_on="ID").set_index('ID_REF')
    gse_annotated.reset_index(level=0, inplace=True)
    gse_annotated.dropna()
    del gse_annotated["ID_REF"]
    del gse_annotated["ID"]
    gse_annotated = gse_annotated.dropna(subset=['GENE_SYMBOL'])#Replace
    gse_annotated = gse_annotated.rename(columns={'GENE_SYMBOL': "Gene"})
    #gse_annotated = gse_annotated[~gse_annotated.Gene.str.contains("///")]
    #gse_annotated['Gene'] = gse_annotated['Gene'].str.split('//').str[1]
    gse_annotated = gse_annotated.groupby("Gene").median()
    if quantile:
        gse_annotated = quantileNormalize(gse_annotated)
    if log:
        gse_annotated = gse_annotated.apply(np.log2, axis=0)
    gse_annotated.reset_index(level=0, inplace=True)
    #print(gse_annotated)
    gse_annotated.to_csv(f'/data/users-workspace/ruchika.sharma/{gse_id}_ExpData.csv', index=False)

In [22]:
error_gse = []
for i, Gid in tqdm(enumerate(unique_GSE)):
    gse = getGEO(Gid)
    try:
        quantile = Gid in raw_signal_GSE
        log = (Gid in quantile_normalized_GSE) or quantile
        prep_deg_input(gse, Gid, quantile, log)
    except (ValueError, KeyError) as e:
        print(Gid, e)
    error_gse.append(Gid)

1it [00:03,  3.73s/it]


In [27]:
GENE_SYMBOL
GB_ACC

['GSE27973',
 'GSE37827',
 'GSE17400',
 'GSE45042',
 'GSE36016',
 'GSE49263',
 'GSE33266',
 'GSE40840',
 'GSE40824',
 'GSE40827',
 'GSE34205',
 'GSE59185',
 'GSE56677',
 'GSE47962',
 'GSE51387',
 'GSE50878',
 'GSE49262',
 'GSE36287',
 'GSE33267',
 'GSE51386',
 'GSE68820']

In [27]:
####Sample_info
df2 = pd.read_csv("/data/users-workspace/ruchika.sharma/sample_ids.csv")
df3 = df2.dropna()
for i in unique_GSE:
    study_specific= df3[df3['ExperimentID'].isin({i})]
    study_specific.to_csv(f'/data/users-workspace/ruchika.sharma/COVID19/Microarray/{i}_Sample_info.csv', index = False)